<a href="https://colab.research.google.com/github/Vlad-ku/colab/blob/main/DFL_me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Монтирование облака
необходимо дать разрешения

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Установка

In [ ]:
# Установка conda (аналог pip и env)

!wget -O cnd.sh https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x cnd.sh
!./cnd.sh -b -f -p /usr/local

In [ ]:
# Установка DFL

%%bash
conda create -n deepfacelab -c main python=3.7 cudnn=7.6.5 cudatoolkit=10.1.243 --yes
eval "$(conda shell.bash hook)" # без этого "conda activate" не работает (и без %%bash тоже)
conda activate deepfacelab
git clone --depth 1 https://github.com/nagadit/DeepFaceLab_Linux.git
cd DeepFaceLab_Linux
git clone --depth 1 https://github.com/iperov/DeepFaceLab.git
pip install -r DeepFaceLab/requirements-colab.txt

In [ ]:
# Фикс скриптов DFL для работы "conda activate"

name_file = '/content/DeepFaceLab_Linux/scripts/env.sh'
target_str = 'conda activate deepfacelab'
append_str = 'eval "$(conda shell.bash hook)" # мой фикс\n'

with open(name_file, 'r') as f:
  old_data = f.read()
if old_data.find('мой фикс') == -1:
  new_data = old_data.replace(target_str, append_str + target_str)
  with open (name_file, 'w') as f:
    f.write(new_data)
  print('пофикшено')
else:
  print('фикс уже был, пропущено')

In [4]:
# Создание рабочей папки

!cd DeepFaceLab_Linux/scripts ; bash 1_clear_workspace.sh

In [6]:
# Копирование исходников в рабочую папку

!cp drive/MyDrive/DFL_src/* DeepFaceLab_Linux/workspace/

# Запуск

In [ ]:
# Нарезка донора на картинки

!cd DeepFaceLab_Linux/scripts ; (echo '0' ; echo 'png') | bash 2_extract_image_from_data_src.sh

# [0] Enter FPS ( ?:help ) : 0
# [png] Output image format ( png/jpg ?:help ) : png

In [ ]:
# Нарезка целевой сцены на картинки

!cd DeepFaceLab_Linux/scripts ; (echo 'png') | bash 3_extract_image_from_data_dst.sh
!cd DeepFaceLab_Linux/scripts ; (echo '7') | bash 3.1_denoise_data_dst_images.sh

# [png] Output image format ( png/jpg ?:help ) : png
# [7] Denoise factor? ( 1-20 ) : 7

In [ ]:
# Извлечение лиц с донора

!cd DeepFaceLab_Linux/scripts ; (echo '0' ; echo 'wf' ; echo '0' ; echo '' ; echo '90' ; echo 'n') | bash 4_data_src_extract_faces_S3FD.sh

# [0] Which GPU indexes to choose? : 0
# [wf] Face type ( f/wf/head ?:help ) : wf
# [0] Max number of faces from image ( ?:help ) : 0
# [512] Image size ( 256-2048 ?:help ) : 512                                    в прошлый раз данные были вроде другие (768)
# [90] Jpeg quality ( 1-100 ?:help ) : 90
# [n] Write debug images to aligned_debug? ( y/n ) : n

In [ ]:
# Загрузка предобученой модели (но это не точно)

!cd DeepFaceLab_Linux/scripts ; bash 4.1_download_CelebA.sh

In [ ]:
# Сортировка лиц донора (видимо не все попадают в обучение, поэтому сортировка по качеству)

!cd DeepFaceLab_Linux/scripts ; (echo '5') | bash 4.2_data_src_sort.sh

# Choose sorting method:
# [0] blur
# [1] motion_blur
# [2] face yaw direction
# [3] face pitch direction
# [4] face rect size in source image
# [5] histogram similarity
# [6] histogram dissimilarity
# [7] brightness
# [8] hue
# [9] amount of black pixels
# [10] original filename
# [11] one face in image
# [12] absolute pixel difference
# [13] best faces
# [14] best faces faster
#
# [5] : 5

In [ ]:
# Извлечение лиц с целевой сцены

!cd DeepFaceLab_Linux/scripts ; (echo '0' ; echo 'wf' ; echo '' ; echo '90') | bash 5_data_dst_extract_faces_S3FD.sh

# [0] Which GPU indexes to choose? : 0
# [wf] Face type ( f/wf/head ?:help ) : wf
# [512] Image size ( 256-2048 ?:help ) : 512                                    в прошлый раз данные были другие (768)
# [90] Jpeg quality ( 1-100 ?:help ) : 90

In [ ]:
# Сортировка лиц с целевой сцены

!cd DeepFaceLab_Linux/scripts ; (echo '5') | bash 5.2_data_dst_sort.sh

# Choose sorting method:
# [0] blur
# [1] motion_blur
# [2] face yaw direction
# [3] face pitch direction
# [4] face rect size in source image
# [5] histogram similarity
# [6] histogram dissimilarity
# [7] brightness
# [8] hue
# [9] amount of black pixels
# [10] original filename
# [11] one face in image
# [12] absolute pixel difference
# [13] best faces
# [14] best faces faster
#
# [5] : 5

# Тренировка
необходимо остановить в ручную

In [ ]:
# Непосредственно обучение модели (советуют держать до 25к)

#!cd DeepFaceLab_Linux/scripts ; bash 6_train_Quick96.sh
!cd DeepFaceLab_Linux/scripts ; (echo 'new' ; echo '0') | bash 6_train_Quick96_no_preview.sh

# [new] No saved models found. Enter a name of a new model : new
# [0] Which GPU indexes to choose? : 0

# Сборка

In [ ]:
# Трансплантация лица

!cd DeepFaceLab_Linux/scripts ; (echo '0\n0\n1\n1\n0\n0\n0\n0\nlct\n0\n0\n0\n0\n0\n8') | bash 7_merge_Quick96.sh

#-------------------------------------------------------------------------------
# Choose one of saved models, or enter a name to create a new model.
# [r] : rename
# [d] : delete
# [0] : new - latest
#  : 0
#-------------------------------------------------------------------------------
# [0] Which GPU indexes to choose? : 0
#-------------------------------------------------------------------------------
# Choose mode:
# (0) original
# (1) overlay
# (2) hist-match
# (3) seamless
# (4) seamless-hist-match
# (5) raw-rgb
# (6) raw-predict
# [1] : 1
#-------------------------------------------------------------------------------
# Choose mask mode:
# (0) full
# (1) dst
# (2) learned-prd
# (3) learned-dst
# (4) learned-prd*learned-dst
# (5) learned-prd+learned-dst
# (6) XSeg-prd
# (7) XSeg-dst
# (8) XSeg-prd*XSeg-dst
# (9) learned-prd*learned-dst*XSeg-prd*XSeg-dst
# [1] : 1
#-------------------------------------------------------------------------------
# [0] Choose erode mask modifier ( -400..400 ) : 0
# [0] Choose blur mask modifier ( 0..400 ) : 0
# [0] Choose motion blur power ( 0..100 ) : 0
# [0] Choose output face scale modifier ( -50..50 ) : 0
# Color transfer to predicted face ( rct/lct/mkl/mkl-m/idt/idt-m/sot-m/mix-m ) : lct
#-------------------------------------------------------------------------------
# Choose sharpen mode:
# (0) None
# (1) box
# (2) gaussian
# [0] ( ?:help ) : 0
#-------------------------------------------------------------------------------
# [0] Choose super resolution power ( 0..100 ?:help ) : 0
# [0] Choose image degrade by denoise power ( 0..500 ) : 0
# [0] Choose image degrade by bicubic rescale power ( 0..100 ) : 0
# [0] Degrade color power of final image ( 0..100 ) : 0
# [8] Number of workers? ( 1-2 ?:help ) : 8

In [ ]:
# Преобразование результата в конечный файл

!cd DeepFaceLab_Linux/scripts ; (echo '16') | bash 8_merged_to_mp4.sh # 16

# [16] Bitrate of output file in MB/s : 16